### **Imports**

In [ ]:
from __future__ import annotations

from pathlib import Path

import pandas as pd

from kub.course.plotlib.simulationPlotFactory import SimulationPlotFactory
from kub.course.plotlib.weatherPlotFactory import WeatherPlotFactory
from kub.course.simlib.simulation import FMUSimulation

repo_root = Path.cwd().parent.parent
databasePath = repo_root / "database" / "day2"

# **Thermal Building Optimization Exercise 1**

Your goal is to optimize the building design parameters (geometry, orientation, and material properties) to **minimize total energy consumption** (heating and cooling) while maintaining thermal comfort within the habitable zone.

***

##  Learning Objectives

Upon completion of this exercise, you should be able to:

1.  **Analyze Building Physics:** Understand the relationship between **thermal inertia** (heavy vs. light materials) placement and building performance under transient conditions.

2.  **Evaluate Envelope Design:** Quantify the influence of **geometry, orientation, and surface absorptivity** on solar gains and heat losses.

3.  **Optimize Energy Consumption:** Determine the most effective strategies (material choice, air change rate, insulation thickness) to minimize HVAC energy demand (`energyConsumptionTotal`).

4.  **Interpret Simulation Results:** Connect the physical phenomena (conduction, convection, radiation) to the simulated heat fluxes (Graph 5) and coefficients (Graph 4).

***

## 1. Building Configuration and Constraints

The following constraints must be respected when setting the parameters in your Python notebook.

### Geometric Constraints

* **Ground Floor Area:** Must be $100 \, \text{m}^2$.
    * $LengthWall1 \times LengthWall2 = 100 \, \text{m}^2$
* **Height:** Must be between $2.7 \, \text{m}$ and $3.0 \, \text{m}$.
    * $2.7 \, \text{m} \le height \le 3.0 \, \text{m}$
* **Wall Orientation (Azimuth):**
  * $0^\circ$ corresponds to the **South** direction.
    * The wall-to-azimuth mapping is constrained in the FMU by the azimuth of wall1 in order to form a box shaped building :
        * `wall_1_floor_1`: `azimuthWall1` (South)
        * `wall_2_floor_1`: `azimuthWall1` $+ 90^\circ$ (West)
        * `wall_3_floor_1`: `azimuthWall1` $+ 180^\circ$ (North)
        * `wall_4_floor_1`: `azimuthWall1` $+ 270^\circ$ (East)

### Material and Construction Constraints

* **Layer Structure:** Every opaque element (Walls, Roof, Bottom) must consist of **two layers**: an interior (`Int`) and an exterior (`Ext`) layer.
    * You must use **one heavy material** (high $\rho$) and **one light material** (low $\rho$) for the two layers of each element using the provided dictionary of materials.
* **Maximum Total Thickness:** The combined thickness of the two layers cannot exceed $40 \, \text{cm}$ ($0.40 \, \text{m}$) for any element.
    * $eIntWall + eExtWall \le 0.40 \, \text{m}$
    * $eIntRoof + eExtRoof \le 0.40 \, \text{m}$
    * $eIntBottom + eExtBottom \le 0.40 \, \text{m}$

***

## 2. Assignment: Optimization Task

Your main goal is to find the combination of parameters that leads to the **lowest `energyConsumptionTotal`** over the simulation period, while maintaining thermal comfort ($T_{setHeating} = 20^\circ\text{C}$, $T_{setCooling} = 24^\circ\text{C}$).

### Suggested Investigation Steps:

1.  **Baseline Simulation:**
    * Choose starting values (e.g., $L_1=10, L_2=10, h=3, \text{azimuth}=0$).

    * Set initial Power Limits (`coolingPower`, `heatingPower`) for the ideal loads (Graph 2).

    * **Monitor:** `energyConsumptionTotal` and `building.zone_habitable_1.TAir` (Graph 1).

2.  **Exterior Surface and Orientation Study:**
    * **Vary `azimuthWall1`** (e.g., $0^\circ, 90^\circ, 180^\circ$) to analyze solar exposure.
    * **Test the extremes of Absorptivity** (`absWallX`, `absRoof`) in relation to the wall's orientation.

        * **Low $\alpha$ (e.g., $0.2$):** Typical for light, reflective surfaces.
        * **High $\alpha$ (e.g., $0.9$):** Typical for dark, non-reflective surfaces.
    * **Analysis:** Observe the impact on solar heat gain by comparing `building.surfacesToAmbience.surface[x].toSurfacesPort.heatPortSw.Q_flow` (Solar Radiation) for walls (Graph 5).

3.  **Material and Thermal Inertia Study:**
    * **Thermal Inertia Placement:** Compare the effects of placing the **heavy material** (high $\rho$) on the **interior** (high thermal inertia) versus the **exterior** layer.

    * **Material Choice:** Optimize the thermal conductivity ($\lambda$) and density ($\rho$) of the two layers for each element (Wall, Roof, Bottom).

    * **Analysis:** Monitor the **Conduction flux** (`building.wall_x_floor_x.construction.heatPort_x2.Q_flow`) for a wall (Graph 5) to see how quickly heat penetrates the construction.

4.  **Air and Ground Study:**
    * **Air Tightness:** Compare energy consumption for **Low** vs **High** values of `airChange`.
        * **Low `airChange` ($0.2 \, \text{AC/h}$):** Airtight building.
        * **High `airChange` ($1.0 \, \text{AC/h}$):** Leaky building.
    * **Ground Coupling:** Test the impact of varying `Tground.T` (Ground Temperature, input in Kelvin) on the required energy.
        * **Low `Tground.T` ($273.15 + 6^\circ\text{C}$):** Representative of winter ground conditions.
        * **High `Tground.T` ($273.15 + 15^\circ\text{C}$):** Representative of summer ground conditions.

### Analysis of Heat Fluxes (Graph 5)

Pay close attention to the **sign convention** when analyzing the fluxes (Graph 5):

* **Positive Fluxes** ($Q_{flow}$) typically indicate heat **entering** the wall or the zone.
* **Negative Fluxes** ($Q_{flow}$) typically indicate heat **leaving** the wall or the zone.

**Key Fluxes to Focus On:**

* `building.surfacesToAmbience.surface[x].toConstructionPort.heatPort.Q_flow`: This is the **total heat flux** exchanged between the exterior surface and the environment (Sum of Convection, Solar Radiation, and Longwave Radiation on the exterior face).

* `building.wall_x_floor_x.construction.heatPort_x2.Q_flow`: This represents the heat **conduction** across the exterior layer, which drives the heat into or out of the building envelope mass.

***

### Success Criteria:

* The final configuration must adhere to **all geometric and material constraints**.
* The final configuration must **minimize `energyConsumptionTotal`** over the simulation period.

Keep a comparison of `energyConsumptionTotal` between your initial baseline simulation and the final optimized result.


# Exercise resolution: ***TO DO***

### **Weather Display**

In [ ]:
# Load Weather Data
df_weather = pd.read_csv(databasePath / "YearlyWeather" / "Strasbourg2023.csv")
df_weather["time"] = pd.to_datetime(df_weather["time"], unit="ms")
df_weather.set_index("time", inplace=True)

weather_factory = WeatherPlotFactory()

weather_factory.plot_yearly_temperature(df_weather)
# weather_factory.plot_yearly_solar_radiation(df_weather)

### **Building thermal simulation**

#### Functions

In [ ]:
def calculate_wall_resistance(Area, hi, he, material_layers):
    """
    Calculates the equivalent thermal resistance (Req) of a wall for N layers.

    Parameters:
    - Area (float): Surface area of the wall (m^2).
    - hi (float): Internal convective heat transfer coefficient (W/(m^2*K)).
    - he (float): External convective heat transfer coefficient (W/(m^2*K)).
                  NOTE: For ground/soil contact walls, 'he' should be set to 0.0.
    - material_layers (list of dict): List of the wall's layers.
      Each dict must contain 'e' (thickness in m) and 'lambda' (conductivity in W/(m*K)).

    Returns:
    - Req (float): The equivalent thermal resistance (K/W).
    """

    # --- 1. Calculate Internal Convective Resistance (R_hi) ---
    # Be careful to handle the case where (hi * Area) might be zero (for R_hi = 0).
    # TODO
    R_hi = ...

    # --- 2. Calculate Total Conduction Resistance (sum of N layers) ---
    R_total_conduction = 0.0
    # Iterate through the material_layers list
    for layer in material_layers:
        thickness = layer["e"]
        lambda_mat = layer["lambda"]
        # Calculate R_conduction for the current layer and add it to R_total_conduction
        # Be careful to handle the case where (lambda_mat * Area) might be zero (for R_cond = 0).
        # TODO
        R_total_conduction += ...

    # --- 3. Calculate External Convective Resistance (R_he) ---
    # Be careful to handle the case where (he * Area) might be zero (for R_he = 0).
    # TODO
    R_he = ...

    # --- 4. Calculate Total Equivalent Resistance (R_eq) ---
    # TODO
    R_eq = ...

    return R_eq

In [ ]:
# test your function using this example
test_layers = [
    {"e": 0.11, "lambda": 0.72}  # Brick layer
]
# uncomment the function call when you're ready
# Req_test = calculate_wall_resistance(Area=1.0, hi=8.0, he=25.0, material_layers=test_layers)
# print(f"Equivalent Resistance: {Req_test:.3f} K/W")

# Expected Output (approximately): 0.354 K/W

In [ ]:
def calculate_total_heat_flow_balance(
    wall_balance_data, T_int, T_ext, airChange, L1, L2, H
):
    """
    Calculates the heat flow (Q_flow) for each wall and the total flow, including ventilation.

    Parameters:
    - wall_balance_data (dict): Dictionary containing the necessary data for the balance.
      Each key is the wall name, and the value is a dict containing:
      'R_eq' (Equivalent Resistance) and 'Delta_T' (T_int - T_target_surface).
    - T_int (float): Reference internal temperature (°C).
    - T_ext (float): Reference external temperature for ventilation (°C).
    - airChange (float): Air change rate (vol/s).
    - L1, L2, H (float): Building dimensions for volume calculation (m).

    Returns:
    - (dict): Individual flows, Q_walls_total, Q_vent, and Q_flow_total.
    """

    # --- Constants for Ventilation ---
    rho_air = 1.2  # Air density (kg/m^3)
    cp_air = 1006.0  # Specific heat capacity of air (J/(kg*K))

    # --- 1. Calculate Heat Loss through Walls (Transmission) ---
    results = {}
    Q_walls_total = 0.0

    for wall_name, data in wall_balance_data.items():
        R_eq = data["R_eq"]
        Delta_T = data["Delta_T"]

        # Calculate Heat Flow (Q_flow = Delta_T / R_eq)
        # Note: Handle the case where R_eq is 0 to avoid division by 0.
        # TODO
        Q_flow = ...

        # Store result for this specific wall
        results[f"Q_flow_{wall_name}"] = Q_flow

        # Add the current wall's Q_flow to the total wall heat loss
        # TODO
        Q_walls_total += ...

    # --- 2. Calculate Heat Loss through Ventilation ---

    # Calculate the building volume based on dimensions (L1, L2, H)
    # TODO
    Volume = ...

    # --- Pre-calculated flow rates (Physics) ---
    V_dot_air = airChange * Volume  # Volumetric flow rate (m^3/s)
    m_dot_air = rho_air * V_dot_air  # Mass flow rate (kg/s)
    Delta_T_vent = T_int - T_ext  # Ventilation is always T_int - T_ext

    # Calculate Q_vent using the formula: Q = m_dot * cp * Delta_T
    # TODO
    Q_vent = ...

    # --- 3. Final Result ---

    # Calculate the total heat flow (Sum of walls + ventilation)
    # TODO
    Q_flow_total = ...

    results["Q_walls_total"] = Q_walls_total
    results["Q_vent"] = Q_vent
    results["Q_flow_total"] = Q_flow_total

    return results

#### Simulation and graph display

##### Names of variables to use in the code

Building Elements correspond to the **walls**, **roof** and the **ground slab** of the Building.

**Building Elements Thickness:**
- Wall
  - inner layer wall thickness : `eIntWall`
  - outer layer wall thickness : `eExtWall`
- Roof
  - inner layer roof thickness : `eIntRoof`
  - outer layer roof thickness : `eExtRoof`
- Ground Slab
  - inner layer bottom thickness : `eIntBottom`
  - outer layer bottom thickness : `eExtBottom`

**Materials properties of Building Elements:**
- Wall
  - thermal conductivity of inner layer (wall) : `lambdaIntWall`
  - thermal conductivity of outer layer (wall) : `lambdaExtWall`
- Roof
  - thermal conductivity of inner layer (roof) : `lambdaIntRoof`
  - thermal conductivity of outer layer (roof) : `lambdaExtRoof`
- Ground Slab
  - thermal conductivity of inner layer (bottom) : `lambdaIntBottom`
  - thermal conductivity of outer layer (bottom) : `lambdaExtBottom`

**Wall length:**
- length of walls 1 and 3 (symmetrical) : `LengthWall1`
- length of walls 2 and 4 (symmetrical) : `LengthWall2`

**Wall height:** `Height`

**Wall absorbtion:**
- absorption wall 1 : `absWall1`
- absorption wall 2 : `absWall2`
- absorption wall 3 : `absWall3`
- absorption wall 4 : `absWall4`

**Indoor air renewal:** `airchange`

**Sizing of heating and cooling capacity**
- heating power : `building.zone_habitable_1.Q_flow_heatingMax`
- cooling power : `building.zone_habitable_1.Q_flow_coolingMax`

In [ ]:
# input variables
inputs = {
    # ---------- WALL PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntWall": 1,
    "eExtWall": 1,
    "lambdaIntWall": 1,
    "lambdaExtWall": 1,
    "rhoIntWall": 1,
    "rhoExtWall": 1,
    "cIntWall": 1,
    "cExtWall": 1,
    # ---------- ROOF PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntRoof": 1,
    "eExtRoof": 1,
    "lambdaIntRoof": 1,
    "lambdaExtRoof": 1,
    "rhoIntRoof": 1,
    "rhoExtRoof": 1,
    "cIntRoof": 1,
    "cExtRoof": 1,
    # ---------- BOTTOM/GROUND PROPERTIES (INTERIOR AND EXTERIOR LAYERS)
    # (thickness, thermal conductivity, density, specific heat)
    "eIntBottom": 1,
    "eExtBottom": 1,
    "lambdaIntBottom": 1,
    "lambdaExtBottom": 1,
    "rhoIntBottom": 1,
    "rhoExtBottom": 1,
    "cIntBottom": 1,
    "cExtBottom": 1,
    # ---------- GEOMETRY, ORIENTATION, AND AIR CHANGE
    # (dimensions of the thermal zone and wall orientation)
    "LengthWall1": 10,
    "LengthWall2": 10,
    "height": 1,
    # ---------- SURFACE ABSORPTIVITY
    "absWall1": 1,
    "absWall2": 1,
    "absWall3": 1,
    "absWall4": 1,
    "absRoof": 1,
    # ---------- HEATING AND COOLING POWER + AIRCHANGE
    "building.zone_habitable_1.Q_flow_heatingMax": 100,  # heatingPower
    "building.zone_habitable_1.Q_flow_coolingMax": -100,  # coolingPower
    "airChange": 1,
}

In [ ]:
# variables to display
temperatureList1 = ["ambience.TAirRef", "building.zone_habitable_1.TAir"]

energyList1 = [
    "energyConsumptionHeating",
    "energyConsumptionCooling",
    "energyConsumptionTotal",
]

convectiveCoeffList1 = ["Wall1_he", "Wall2_he", "Wall3_he", "Wall4_he", "Roof_he", "hi"]

QflowExteriorSurfacesList1 = [
    "building.surfacesToAmbience.surface[1].toConstructionPort.heatPort.Q_flow",
    "building.surfacesToAmbience.surface[2].toConstructionPort.heatPort.Q_flow",
    "building.surfacesToAmbience.surface[3].toConstructionPort.heatPort.Q_flow",
    "building.surfacesToAmbience.surface[4].toConstructionPort.heatPort.Q_flow",
    "building.surfacesToAmbience.surface[5].toConstructionPort.heatPort.Q_flow",
]

QflowConductionExteriorSurfacesList1 = [
    "building.wall_1_floor_1.construction.heatPort_x2.Q_flow",
    "building.wall_1_floor_1.construction.heatPort_x2.Q_flow",
    "building.wall_1_floor_1.construction.heatPort_x2.Q_flow",
    "building.wall_1_floor_1.construction.heatPort_x2.Q_flow",
    "building.roof_panel_1.construction.heatPort_x2.Q_flow",
    "building.bottom.construction.heatPort_x2.Q_flow",
]

In [ ]:
# simulation
sim1 = FMUSimulation(
    databasePath
    / "BuildingModels"
    / "Exercices_Optimization_LoD0Bui4Walls1Floor1Roof_App4Walls1Floor1Roof.fmu"
)

sim1.initialize(startTime=0.0, stopTime=86400 * 365, timeStep=3600.0)
sim1.initParameters(inputs)  # put inputs in the function when you're ready
sim1.exitInitialization()

data1 = sim1.run(
    temperatureList1
    + convectiveCoeffList1
    + energyList1
    + QflowExteriorSurfacesList1
    + QflowConductionExteriorSurfacesList1
)

sim1Factory = SimulationPlotFactory()

In [ ]:
# Plot: evolution of temperature
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="temperature",
    data_dict={
        "Exterior Temperature (°C)": data1["ambience.TAirRef"],
        "Interior Temperature (°C)": data1["building.zone_habitable_1.TAir"],
    },
    title="Evolution of Temperature",
)

In [ ]:
# Plot: convective coefficients
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="raw",
    data_dict={
        "Wall1_he": data1["Wall1_he"],
        "Wall2_he": data1["Wall2_he"],
        "Wall3_he": data1["Wall3_he"],
        "Wall4_he": data1["Wall4_he"],
        "Roof_he": data1["Roof_he"],
        "hi": data1["hi"],
    },
    title="Convective Coefficients",
)

In [ ]:
# Plot: energy consumption
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="energy",
    data_dict={
        "Cooling": data1["energyConsumptionCooling"],
        "Heating": data1["energyConsumptionHeating"],
        "Total": data1["energyConsumptionTotal"],
    },
    title="Energy Consumption over the year",
)

In [ ]:
# Plot: Total Heat Flux exchanged between the exterior surface and the environment
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={
        "Wall 1": data1[
            "building.surfacesToAmbience.surface[1].toConstructionPort.heatPort.Q_flow"
        ],
        "Wall 2": data1[
            "building.surfacesToAmbience.surface[2].toConstructionPort.heatPort.Q_flow"
        ],
        "Wall 3": data1[
            "building.surfacesToAmbience.surface[3].toConstructionPort.heatPort.Q_flow"
        ],
        "Wall 4": data1[
            "building.surfacesToAmbience.surface[4].toConstructionPort.heatPort.Q_flow"
        ],
        "Roof": data1[
            "building.surfacesToAmbience.surface[5].toConstructionPort.heatPort.Q_flow"
        ],
    },
    title="Total Heat Flux exchanged between the exterior surface and the environment (Sum of Convection, Solar Radiation, and Longwave Radiation on the exterior face)",
)

In [ ]:
# Plot: heat conduction across the exterior layer
sim1Factory.plot_multi_curves(
    time=data1["time"],
    data_type="heat_flux",
    data_dict={
        "Wall 1": data1["building.wall_1_floor_1.construction.heatPort_x2.Q_flow"],
        "Wall 2": data1["building.wall_1_floor_1.construction.heatPort_x2.Q_flow"],
        "Wall 3": data1["building.wall_1_floor_1.construction.heatPort_x2.Q_flow"],
        "Wall 4": data1["building.wall_1_floor_1.construction.heatPort_x2.Q_flow"],
        "Roof": data1["building.roof_panel_1.construction.heatPort_x2.Q_flow"],
        "Bottom (Ground)": data1["building.bottom.construction.heatPort_x2.Q_flow"],
    },
    title="Heat Conduction across the exterior layer",
)